<a href="https://colab.research.google.com/github/EdWangLoDaSc/Prob-cGAN-for-Drug-Discovery/blob/main/Kriging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pykrige

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.interpolate import griddata
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import (
    Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Cropping2D,
    AveragePooling2D, Flatten, Reshape, Dropout, Conv2D, LSTM,
    RepeatVector
)
from tensorflow.keras.losses import MSE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity as ssim
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

import h5py
from tqdm import tqdm


from pykrige.ok3d import OrdinaryKriging3D
from pykrige.uk3d import UniversalKriging3D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 3D kriging

In [ ]:
import numpy as np
import h5py
from tqdm import tqdm

def generate_data(f,n, sen_num_kind_list, sen_num_var_list):
    lat = np.array(f['lat'])
    lon = np.array(f['lon'])
    sst = np.array(f['sst'])
    sst1 = np.nan_to_num(sst)
    sst_reshape = sst[0, :].reshape(len(lat[0, :]), len(lon[0, :]), order='F')
    xv1, yv1 = np.meshgrid(lon[0, :], lat[0, :])

    # Adjust X_ki to have 3 columns instead of 2 to include times
    X_ki = np.zeros((1040 * n, 3))
    y_ki = np.zeros((1040 * n, 1))

    for ki in tqdm(range(len(sen_num_kind_list))):
        sen_num = sen_num_kind_list[ki]
        # Also adjust X_va to have 3 columns
        X_va = np.zeros((1040 * n, 3))
        y_va = np.zeros((1040 * n, 1))

        for va in tqdm(range(len(sen_num_var_list))):
            X_t = np.zeros((1040, len(lat[0, :]), len(lon[0, :]), 2))
            y_t = np.zeros((1040, len(lat[0, :]), len(lon[0, :]), 1))
            times = 0
            for t in tqdm(range(1040)):
                y_t[t, :, :, 0] = np.nan_to_num(sst[t, :].reshape(len(lat[0, :]), len(lon[0, :]), order='F'))

                np.random.seed(sen_num_var_list[va])
                sparse_locations_lat = np.random.randint(len(lat[0, :]), size=(sen_num))
                sparse_locations_lon = np.random.randint(len(lon[0, :]), size=(sen_num))

                sparse_locations = np.column_stack((sparse_locations_lat, sparse_locations_lon))
                times += 0.00001
                for s in range(sen_num):
                    a, b = sparse_locations[s]
                    while np.isnan(sst_reshape[int(a), int(b)]):
                        a = np.random.randint(len(lat[0, :]))
                        b = np.random.randint(len(lon[0, :]))
                        sparse_locations[s, 0] = a
                        sparse_locations[s, 1] = b

                sparse_data = np.zeros((sen_num))
                for s in range(sen_num):
                    sparse_data[s] = (y_t[t, :, :, 0][int(sparse_locations[s, 0]), int(sparse_locations[s, 1])])

                # Incorporate times into X_va along with the locations
                X_va[n * t:n * (t + 1), :2] = sparse_locations
                X_va[n * t:n * (t + 1), 2] = times
                y_va[n * t:n * (t + 1), :] = sparse_data.reshape((n, 1))

            X_ki[1040 * n * ki:1040 * n * (ki + 1), :] = X_va
            y_ki[1040 * n * ki:1040 * n * (ki + 1), :] = y_va

    return X_ki, y_ki


f = h5py.File('/content/drive/MyDrive/Physics/Physics/sst_weekly.mat', 'r')

n = 340
sen_num_kind_list = [n]
sen_num_var_list = [900]
X_ki, y_ki = generate_data(f, n,sen_num_kind_list, sen_num_var_list)


In [ ]:
import numpy as np
import time
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
# Assuming you have a working OrdinaryKriging3D implementation available

# Load y_test
y_test = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_test.npy', mmap_mode='r')

# Prepare grid for Kriging interpolation
gridx = np.arange(0.0, 180, 1)
gridy = np.arange(0.0, 360, 1)
gridz = np.arange(0.0, 0.00006, 0.00001)

# Initialize metrics
ssim_vals = []
psnr_vals = []
x_time = X_ki[0:3*n][:, 2]

times = 0
for i in range(0, 30):
    # Reshape and select subsets for Kriging
    X_va_2 = X_ki[i*n:(i+3)*n]
    y_va_2 = y_ki[i*n:(i+3)*n]

    # Perform 3D Kriging
    ok3d_2 = OrdinaryKriging3D(
        X_va_2[:, 0], X_va_2[:, 1], x_time, y_va_2,
        variogram_model="spherical", nlags=8
    )
    start_time = time.time()
    k3d1, ss3d = ok3d_2.execute("grid", gridx, gridy, gridz)
    end_time = time.time()

    print(f"Inference Time for i={i}: ", end_time - start_time, "seconds")
    times+=(end_time - start_time)
    # Apply mask and rotate
    first_element = y_test[0, :, :, :1]
    mask = (first_element == 0)
    result = np.empty_like(k3d1)
    for j in range(k3d1.shape[0]):
        masked_rotated_element = np.where(mask.squeeze(), 0, first_element[:, :, 0])
        masked_rotated_element = np.rot90(masked_rotated_element, k=1)
        result[j] = k3d1[j] + masked_rotated_element

    # Compare to true images and calculate metrics
    true_set = [np.rot90(y_test[i+j, :, :, :1], k=1) for j in range(3, 6)]
    result_set = result[-3:]  # Last 3 images as they correspond to the true_set

    for true, pred in zip(true_set, result_set):
        ssim_vals.append(ssim(true.squeeze(), pred, data_range=pred.max() - pred.min()))
        psnr_vals.append(psnr(true.squeeze(), pred, data_range=pred.max() - pred.min()))

# After the loop
# Calculate average metrics or otherwise summarize them
avg_ssim = np.mean(ssim_vals)
avg_psnr = np.mean(psnr_vals)

print(f"Average SSIM: {avg_ssim}")
print(f"Average PSNR: {avg_psnr}")


# 2D kriging

In [ ]:
import numpy as np
import h5py
from tqdm import tqdm
from scipy.interpolate import griddata

def generate_data(f,n, sen_num_kind_list, sen_num_var_list):
    lat = np.array(f['lat'])
    lon = np.array(f['lon'])
    sst = np.array(f['sst'])
    sst1 = np.nan_to_num(sst)
    sst_reshape = sst[0, :].reshape(len(lat[0, :]), len(lon[0, :]), order='F')
    xv1, yv1 = np.meshgrid(lon[0, :], lat[0, :])

    X_ki = np.zeros((1040 * n, 2))
    y_ki = np.zeros((1040 * n, 1))

    for ki in tqdm(range(len(sen_num_kind_list))):
        sen_num = sen_num_kind_list[ki]
        X_va = np.zeros((1040 * n, 2))
        y_va = np.zeros((1040 * n, 1))

        for va in tqdm(range(len(sen_num_var_list))):
            X_t = np.zeros((1040, len(lat[0, :]), len(lon[0, :]), 2))
            y_t = np.zeros((1040, len(lat[0, :]), len(lon[0, :]), 1))

            for t in tqdm(range(1040)):
                y_t[t, :, :, 0] = np.nan_to_num(sst[t, :].reshape(len(lat[0, :]), len(lon[0, :]), order='F'))

                np.random.seed(sen_num_var_list[va])
                sparse_locations_lat = np.random.randint(len(lat[0, :]), size=(sen_num))
                sparse_locations_lon = np.random.randint(len(lon[0, :]), size=(sen_num))

                sparse_locations = np.column_stack((sparse_locations_lat, sparse_locations_lon))

                for s in range(sen_num):
                    a, b = sparse_locations[s]
                    while np.isnan(sst_reshape[int(a), int(b)]):
                        a = np.random.randint(len(lat[0, :]))
                        b = np.random.randint(len(lon[0, :]))
                        sparse_locations[s, 0] = a
                        sparse_locations[s, 1] = b

                sparse_data = np.zeros((sen_num))
                for s in range(sen_num):
                    sparse_data[s] = (y_t[t, :, :, 0][int(sparse_locations[s, 0]), int(sparse_locations[s, 1])])

                X_va[n * t:n * (t + 1), :] = sparse_locations
                y_va[n * t:n * (t + 1), :] = sparse_data.reshape((n, 1))

            X_ki[1040 * n * ki:1040 * n * (ki + 1), :] = X_va
            y_ki[1040 * n * ki:1040 * n * (ki + 1), :] = y_va

    return X_ki, y_ki


f = h5py.File('/content/drive/MyDrive/Physics/Physics/sst_weekly.mat', 'r')
n=320

sen_num_kind_list = [n]
sen_num_var_list = [900]
X_ki, y_ki = generate_data(f,n ,sen_num_kind_list, sen_num_var_list)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykrige.ok import OrdinaryKriging

def kriging_interpolation(x, y, gridx, gridy):
    """
    Performs Ordinary Kriging interpolation.

    Args:
        x (ndarray): Input x coordinates array.
        y (ndarray): Input y coordinates array.
        n (int): Number of points to select.
        gridx (ndarray): X direction grid points.
        gridy (ndarray): Y direction grid points.

    Returns:
        ndarray: Interpolated values.
    """

    OK = OrdinaryKriging(
        x[:, 0],
        x[:, 1],
        y,
        variogram_model='spherical',
        verbose=True,
        enable_plotting=True,
    )

    zstar, ss = OK.execute("grid", gridx, gridy)

    cax = plt.imshow(zstar, extent=(0, 180, 0, 360))
    cbar = plt.colorbar(cax)
    plt.scatter(x[:, 0], x[:, 1], c='k', marker='.')
    plt.title(f'{n}_Porosity estimate')
    plt.show()

    return zstar
i = 6
k = 3*i
# Sample usage
img_num_0 = k
img_num_1 = k + 1
img_num_2 = k + 2
img_num_3 = k + 3
img_num_4 = k + 4
img_num_5 = k + 5

gridx = np.arange(0, 180, 1, dtype='float64')
gridy = np.arange(0, 360, 1, dtype='float64')
X_va = X_ki.reshape((1040, 200, 2))
y_va = y_ki.reshape((1040, 200, 1))
zstar_0 = kriging_interpolation(X_va[img_num_0], y_va[img_num_0],  gridx, gridy)
zstar_1 = kriging_interpolation(X_va[img_num_1], y_va[img_num_1],  gridx, gridy)
zstar_2 = kriging_interpolation(X_va[img_num_2], y_va[img_num_2],  gridx, gridy)
zstar_3 = kriging_interpolation(X_va[img_num_3], y_va[img_num_3],  gridx, gridy)
zstar_4 = kriging_interpolation(X_va[img_num_4], y_va[img_num_4], gridx, gridy)
zstar_5 = kriging_interpolation(X_va[img_num_5], y_va[img_num_5],  gridx, gridy)


In [ ]:
X_va = X_ki.reshape((1040, n, 2))
y_va = y_ki.reshape((1040, n, 1))

import numpy as np
import matplotlib.pyplot as plt
from pykrige.ok import OrdinaryKriging

def kriging_interpolation(x, y, gridx, gridy):
    """
    Performs Ordinary Kriging interpolation.

    Args:
        x (ndarray): Input x coordinates array.
        y (ndarray): Input y coordinates array.
        n (int): Number of points to select.
        gridx (ndarray): X direction grid points.
        gridy (ndarray): Y direction grid points.

    Returns:
        ndarray: Interpolated values.
    """

    variogram_model = 'spherical'
    '''variogram_parameters = {'sill': 0.1, 'range': 50 , 'nugget': 0.1}  # Example parameters'''

    OK = OrdinaryKriging(
        x[:, 0],
        x[:, 1],
        y,
        variogram_model=variogram_model,
        verbose=True,
        enable_plotting=False,  # Set to False to avoid automatic plotting
        nlags=8
    )

    zstar, ss = OK.execute("grid", gridx, gridy)

    cax = plt.imshow(zstar, extent=(0, 180, 0, 360))
    cbar = plt.colorbar(cax)
    plt.scatter(x[:, 0], x[:, 1], c='k', marker='.')
    plt.title(f'{n}_Porosity estimate')
    plt.show()

    return zstar


y_test = np.load('/content/drive/MyDrive/Physics/Dataset10/y_NOAA_test.npy',mmap_mode = 'r')
# Extract the first element of x_train
first_element = y_test[0][:,:,:1]

# Create a mask for zero values
mask = (first_element == 0)

rotated_element = np.rot90(first_element, k=1)

In [ ]:
from sklearn.linear_model import LinearRegression

true_set = []
predict_set = []
import time
total_time = 0
for i in range(0, 5):
    k = 3 * i
    print(i)
    # Sample usage
    img_num_0 = k
    img_num_1 = k + 1
    img_num_2 = k + 2
    img_num_3 = k + 3

    gridx = np.arange(0, 180, 1, dtype='float64')
    gridy = np.arange(0, 360, 1, dtype='float64')
    X_va = X_ki.reshape((1040, n, 2))
    y_va = y_ki.reshape((1040, n, 1))
    zstar_0 = kriging_interpolation(X_va[img_num_0], y_va[img_num_0],  gridx, gridy)
    zstar_1 = kriging_interpolation(X_va[img_num_1], y_va[img_num_1],  gridx, gridy)
    zstar_2 = kriging_interpolation(X_va[img_num_2], y_va[img_num_2],  gridx, gridy)
    zstar_3 = kriging_interpolation(X_va[img_num_3], y_va[img_num_3],  gridx, gridy)

    # Flatten images into 1D vectors
    X = np.array([zstar_0.flatten(), zstar_1.flatten(), zstar_2.flatten()]).T

    # Target value is the next image zstar_3
    y = zstar_3.flatten()

    # Create linear regression model
    model = LinearRegression()

    # Fit the model
    model.fit(X, y)

    start_time = time.time()

    # Perform prediction for the next three steps
    next_image_1 = model.predict(np.array([zstar_0.flatten(), zstar_1.flatten(), zstar_2.flatten()]).T)
    next_image_2 = model.predict(np.array([zstar_1.flatten(), zstar_2.flatten(), next_image_1]).T)
    next_image_3 = model.predict(np.array([zstar_2.flatten(), next_image_1, next_image_2]).T)

    rotated_element = np.rot90(first_element, k=1).reshape(360, 180, 1)
    next_image_1 = next_image_1.reshape(360, 180, 1)
    next_image_2 = next_image_2.reshape(360, 180, 1)
    next_image_3 = next_image_3.reshape(360, 180, 1)

    # Add the rotated element to next_image_1, next_image_2, and next_image_3
    next_image_1 += rotated_element
    next_image_2 += rotated_element
    next_image_3 += rotated_element

    # End time for the current iteration
    end_time = time.time()

    # Calculate the time taken for the current iteration
    iteration_time = end_time - start_time
    print("Total time taken: ", iteration_time, "minutes")
    # Accumulate the time for all iterations
    total_time += iteration_time

    # Append to true_set and predict_set
    true_pic_1 = y_test[k + 3]
    true_pic_2 = y_test[k + 4]
    true_pic_3 = y_test[k + 5]

    # Rotate true_pic_1 by 90 degrees clockwise
    rotated_true_pic_1 = np.rot90(true_pic_1, k=1)

    # Rotate true_pic_2 by 90 degrees clockwise
    rotated_true_pic_2 = np.rot90(true_pic_2, k=1)

    # Rotate true_pic_3 by 90 degrees clockwise
    rotated_true_pic_3 = np.rot90(true_pic_3, k=1)

    true_set.append([rotated_true_pic_1, rotated_true_pic_2, rotated_true_pic_3])
    predict_set.append([next_image_1, next_image_2, next_image_3])

true_set = np.array(true_set)
predict_set = np.array(predict_set)


In [ ]:
total_time_minutes = total_time / 60  # Convert total_time to minutes
print("Total time taken for all iterations: ", total_time_minutes, "minutes")



1040/30*total_time_minutes*60

In [ ]:
predict_set.shape

In [ ]:
predict_set = predict_set.reshape((15,360,180,1))
true_set = true_set.reshape((15,360,180,1))

In [ ]:
plt.imshow(predict_set[10,:,:,:])

In [ ]:



from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Calculate SSIM and PSNR for the first predicted image
ssim_1 = ssim(true_set, predict_set, multichannel=True)
# Calculate PSNR for the first predicted image
psnr_1 = psnr(true_set, predict_set, data_range=predict_set.max() - predict_set.min())

# Calculate SSIM and PSNR for the second predicted imag
ssim_1